In [61]:
import requests
import os
import json

import pandas as pd
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = ''


def create_url():
    # Replace with user ID below
    user_id = 928759224599040001
    return "https://api.twitter.com/2/users/{}/tweets".format(user_id)

In [19]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserTweetsPython"
    return r

In [48]:
import time
import requests

def connect_to_endpoint(url, params, max_retries=10, wait_seconds=10, rate_limit_wait=900):
    retries = 0
    while retries < max_retries:
        try:
            response = requests.request("GET", url, auth=bearer_oauth, params=params)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print("Rate limit exceeded. Waiting for 15 minutes...")
                time.sleep(rate_limit_wait)  # 특정 시간(예: 15분) 동안 대기
                retries += 1
            else:
                raise Exception("Non-200 status code: {} {}".format(response.status_code, response.text))
        except (requests.ConnectionError, requests.Timeout, requests.RequestException) as e:
            retries += 1
            print(f"Connection error occurred: {e}. Retrying {retries}/{max_retries}...")
            time.sleep(wait_seconds)

    raise Exception("Max retries exceeded")



In [43]:
def get_params(next_token=None):
    params = {
        "tweet.fields": "created_at,text",
        "max_results": 100
    }
    if next_token:
        params["pagination_token"] = next_token
    return params


In [60]:
def main():
    url = create_url()
    next_token = None
    total_tweets = []

    for _ in range(32):  # 최대 3200개의 트윗을 가져오기 위해 32번 반복
        params = get_params(next_token)
        json_response = connect_to_endpoint(url, params)
        tweets = json_response.get('data', [])
        total_tweets.extend(tweets)
        next_token = json_response.get('meta', {}).get('next_token', None)
        if not next_token:
            break

    # 결과 처리
    if total_tweets:
        df = pd.DataFrame(total_tweets)
        df = df[['created_at', 'text']]  # 필요한 컬럼만 선택
        df.to_csv('tweets_BloombergCrypto.csv', index=False)
        print(f"Total tweets fetched: {len(total_tweets)}")
    else:
        print("No tweets fetched.")

In [62]:
main()

Connection error occurred: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/users/928759224599040001/tweets?tweet.fields=created_at%2Ctext&max_results=100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f390e311630>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')). Retrying 1/10...
Connection error occurred: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/users/928759224599040001/tweets?tweet.fields=created_at%2Ctext&max_results=100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f390e310880>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')). Retrying 2/10...
Connection error occurred: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/users/928759224599040001/tweets?tweet.fields=created_at%2Ctext&max_results=100&pagination_to